# Chatbot 

Web-based interface of the chatbot interface: https://gonzalorecio.com/chatbot/robot.html

In [1]:
import requests

In [2]:
class ChatbotFace:
    chatbot_API = 'https://chatbot-mood.herokuapp.com/mood'

    def change_mood(self, mood):
        req_obj = {'action': mood}
        x = requests.post(self.chatbot_API, data = req_obj)
        print(x)
        
    def neutral(self):
        self.change_mood('neutral')
        
    def happy(self):
        self.change_mood('happy')
        
    def sad(self):
        self.change_mood('sad')
        
    def angry(self):
        self.change_mood('angry')
        
    def focused(self):
        self.change_mood('focused')
        
    def confused(self):
        self.change_mood('confused')
    
    def start_blinking(self):
        self.change_mood('start_blinking')
        
    def stop_blinking(self):
        self.change_mood('stopt_blinking')


In [3]:
chatbot = ChatbotFace()

In [4]:
chatbot.confused()

<Response [200]>


In [5]:
chatbot.happy()

<Response [200]>


In [6]:
chatbot.neutral()

<Response [200]>


# Sentiment

In [7]:
from sentiments import sentiment_analysis as sent

Score: {'neg': 0.4, 'neu': 0.408, 'pos': 0.192, 'compound': -0.5499}
I am happy but today my mom died
Negative


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\gonre\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gonre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\gonre\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gonre\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# Test
sent.sentiment_analysis("I've had a nice day today")

Score: {'neg': 0.0, 'neu': 0.588, 'pos': 0.412, 'compound': 0.4215}
I've had a nice day today
Positive


1

## Speech

In [9]:
from speech import google_speech as speech

In [13]:
# Test
speech.speech_to_audio(lang="en-US")

Listening...
Recognizing...
hola hola


'hola hola'

In [10]:
# Test
s = 'what are you saying?? :D'
speech.text_to_speech(s, speed=150, voice=1)

## Translation

In [10]:
from google_trans_new import google_translator  
translator = google_translator()  
translate_text = translator.translate('สวัสดีจีน', lang_tgt='es')  
print(translate_text)
#output: Hello china

Hola china 


# Chatbot

In [11]:
import tensorflow
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from google_trans_new import google_translator  

class Chatbot:
    def __init__(self, lang='en'):
        print('Loading model and tokenizers...')
        self.model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
        self.face = ChatbotFace()
        
        
        
        self.reset_chatbot(lang=lang)
        print('Chatbot ready.')
    
    def reset_chatbot(self, lang='en'):
        self.chat_history_ids = self.tokenizer.encode(self.tokenizer.bos_token, return_tensors='pt')
        if lang=='es':
            self.voice = 0 # 0: spanish female, 1: english female, 2: english male
        else:
            self.voice = 1 # 0: spanish female, 1: english female, 2: english male
        self.translator = google_translator()  
        self.lang = lang
        
    def listen_and_get_question(self):
        lang = 'en-US' if self.lang=='en' else 'es-ES'
        question = speech.speech_to_audio()
        if self.lang=='es':
            question = self.translate(question, lang='en')
        return question
    
    def decode(self, token_ids):
        return self.tokenizer.decode(token_ids, skip_special_tokens=True)
    
    def generate_answer(self, question):
        question_ids = self.tokenizer.encode(question + self.tokenizer.eos_token, return_tensors='pt')
        input_ids = torch.cat([self.chat_history_ids, question_ids], dim=-1)
        self.chat_history_ids = self.model.generate(input_ids, max_length=1000, pad_token_id=self.tokenizer.eos_token_id)
        answer_ids = self.chat_history_ids[:, input_ids.shape[-1]:][0]
        text = self.decode(answer_ids)
        if self.lang == 'es':
            text = self.translate(text, lang='es')
        return text
    
    def get_sentiment_analysis(self, text):
        return True
        
    def speak(self, text):
        self.face.happy()
        speech.text_to_speech(text, speed=150, voice=self.voice)
        self.face.neutral()
        
    def translate(self, text, lang):
        translated_text = self.translator.translate(text, lang_tgt=lang)
        if type(translated_text) == list:
            return translated_text[0]
        return translated_text
    
    def no_understand(self):
        text = "Sorry, I couldn't understand you"
        if self.lang == 'es':
            text = self.translete(text, lang='es')
        self.speak(text)
    
    def run_chat(self):
        self.reset_chatbot(lang=self.lang)
        self.face.neutral()
        question = ''
        while(question != 'goodbye'):
            question = self.listen_and_get_question()
            if question is None:
                print('Fallo')
                self.no_understand()
                continue
            print('Question:', question)
            answer = self.generate_answer(question)
            print('Answer:', answer)
            self.speak(answer)
            
#             self.reset_chatbot(lang=self.lang)
            print(self.chat_history_ids)
        

In [12]:
chatbot = Chatbot(lang='es')
# chatbot.run()

Loading model and tokenizers...
Chatbot ready.


In [15]:
chatbot.reset_chatbot(lang='en')

In [16]:
chatbot.run_chat()

<Response [200]>
Listening...
Recognizing...
hello hello
Question: hello hello
Answer: Hello hello
<Response [200]>
<Response [200]>
tensor([[50256, 31373, 23748, 50256, 15496, 23748, 50256]])
Listening...
Recognizing...
si ahora llamaste a santi no han abierto las
Question: si ahora llamaste a santi no han abierto las
Answer: Si alegro
<Response [200]>
<Response [200]>
tensor([[50256, 31373, 23748, 50256, 15496, 23748, 50256, 13396, 29042,  5799,
         32660,   321,  4594,   257,   264, 17096,   645,   289,   272,   450,
            72, 13806, 39990, 50256, 42801,   257,  1455,   305, 50256]])
Listening...
Recognizing...
ahora se llamaste a santi no han abierto las
Question: ahora se llamaste a santi no han abierto las
Answer: Si alegro
<Response [200]>
<Response [200]>
tensor([[50256, 31373, 23748, 50256, 15496, 23748, 50256, 13396, 29042,  5799,
         32660,   321,  4594,   257,   264, 17096,   645,   289,   272,   450,
            72, 13806, 39990, 50256, 42801,   257,  1455,

KeyboardInterrupt: 